In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric
from sqlalchemy.orm import sessionmaker
import pandas as pd

In [3]:
import sqlalchemy
from sqlalchemy import create_engine, MetaData

# Dependencies and boilerplate
from sqlalchemy import Column, Float, Integer, String
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()

In [4]:
# Use a Session to test the Prescription class
### BEGIN SOLUTION
from sqlalchemy import create_engine
from sqlalchemy.orm import Session

engine = create_engine("sqlite:///hawaii.sqlite")
Base.metadata.create_all(engine)

session = Session(bind=engine)
### END SOLUTION

In [5]:
# Define a Measurement class
### BEGIN SOLUTION
class Measurements(Base):
    __tablename__ = 'measurements'
    id = Column(Integer, primary_key=True,autoincrement=True)
    station = Column(String(255))
    date= Column(String(10))
    prcp = Column(Float)
    tobs = Column(Integer)

class Stations(Base):
    __tablename__ = 'stations'
    station = Column(String(255), primary_key=True)
    name = Column(String(255))
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)
### END SOLUTION

In [6]:
#create the table in db 
Base.metadata.create_all(engine)

In [7]:
Base.metadata.tables

immutabledict({'measurements': Table('measurements', MetaData(bind=None), Column('id', Integer(), table=<measurements>, primary_key=True, nullable=False), Column('station', String(length=255), table=<measurements>), Column('date', String(length=10), table=<measurements>), Column('prcp', Float(), table=<measurements>), Column('tobs', Integer(), table=<measurements>), schema=None), 'stations': Table('stations', MetaData(bind=None), Column('station', String(length=255), table=<stations>, primary_key=True, nullable=False), Column('name', String(length=255), table=<stations>), Column('latitude', Float(), table=<stations>), Column('longitude', Float(), table=<stations>), Column('elevation', Float(), table=<stations>), schema=None)})

In [8]:
#Create dataframes with files
hmeasurements= pd.read_csv('C:\\Pri doc\\priti\\Projects\\Myproject\\Berkely\\Week10SQL\\clean_hawaii_measurements.csv')
hstations= pd.read_csv('C:\\Pri doc\\priti\\Projects\\Myproject\\Berkely\\Week10SQL\\clean_hawaii_stations.csv')

In [9]:
# The orient='records' is the key of this, it allows to align with the format mentioned in the doc to insert in bulks.
measurements_data = hmeasurements.to_dict(orient='records')
stations_data = hstations.to_dict(orient='records')


In [10]:
# Use MetaData from SQLAlchemy to reflect the tables
# YOUR CODE HERE
metadata = MetaData(bind=engine)
metadata.reflect()

In [11]:
measurements_table = sqlalchemy.Table('measurements', metadata, autoload=True)
stations_table = sqlalchemy.Table('stations', metadata, autoload=True)

In [12]:
conn = engine.connect()
conn.execute(measurements_table.delete())
conn.execute(stations_table.delete())

In [13]:
conn.execute(measurements_table.insert(),measurements_data)
conn.execute(stations_table.insert(),stations_data)

In [14]:
#checking if all records count
measurement_count = session.query(Measurements.id,Measurements.station).count()
measurement_count

19550

In [15]:
station_count = session.query(Stations.station).count()

station_count

9

In [16]:
#Check the records in the tables
conn.execute("select * from Stations ").fetchall()

[('USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 ('USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 ('USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0),
 ('USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 ('USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6),
 ('USC00519523', 'WAIMANALO EXPERIMENTAL FARM, HI US', 21.33556, -157.71139, 19.5),
 ('USC00519281', 'WAIHEE 837.5, HI US', 21.45167, -157.84888999999995, 32.9),
 ('USC00511918', 'HONOLULU OBSERVATORY 702.2, HI US', 21.3152, -157.9992, 0.9),
 ('USC00516128', 'MANOA LYON ARBO 785.2, HI US', 21.3331, -157.8025, 152.4)]